# Homework 4

**Name:** -- Víctor Manuel Mariscal Cervantes --

**e-mail:** -- victor.mariscal4459@alumnos.udg.mx --

# MODULES

In [2]:
# Load modules
import numpy as np
import math
import scipy.stats as stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import panel as pn
import param
import pandas as pd
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
import hvplot.pandas


In [3]:
# Enable Panel extensions
pn.extension('plotly')
hv.extension('plotly')


/tmp/ipykernel_23623/88948998.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('plotly')


In [4]:
# Define Vec2d class for vector operations
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # String representation
    def __str__(self):
        return f"({self.x}, {self.y})"
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # Rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
    # Return as tuple
    def as_tuple(self):
        return (self.x, self.y)

# Randon Walk Trajectory Generation functions

In [5]:
# Brownian Motion
def generate_bm_trajectory(num_steps, speed, start_pos_x, start_pos_y, seed=None):
    """
    Generate a Brownian Motion trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample random direction (uniform in all directions)
        angle = np.random.uniform(0, 2 * np.pi)
        
        # Create step vector
        step = Vec2d(np.cos(angle) * speed, np.sin(angle) * speed)
        
        # Update position
        current_pos = current_pos + step
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

In [6]:
# Correlated Random Walk
def generate_crw_trajectory(num_steps, speed, start_pos_x, start_pos_y, cauchy_coef, seed=None):
    """
    Generate a Correlated Random Walk trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient (0 < c < 1)
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction and apply speed
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length * speed, direction.y / length * speed)
        
        # Update position
        current_pos = current_pos + direction
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

In [7]:
# Lévy Flight
def generate_lf_trajectory(num_steps, speed, start_pos_x, start_pos_y, cauchy_coef, levy_exponent, seed=None):
    """
    Generate a Lévy Flight trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Base speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient (0 < c < 1)
    levy_exponent : float
        Lévy exponent (alpha) controlling the power law distribution (1 < alpha < 3)
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Step length from Lévy distribution (power law)
        # We use the Pareto distribution which is a power law
        # The exponent mu = 1 + alpha 
        step_length = np.random.pareto(levy_exponent) * speed
        
        # Normalize direction and apply step length
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length * step_length, 
                              direction.y / length * step_length)
        
        # Update position
        current_pos = current_pos + direction
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

# Metric Calculation Functions

In [8]:
# Path Length
def calculate_pl(trajectory):
    """
    Calculate cumulative Path Length for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    time : numpy.ndarray
        Time points
    pl : numpy.ndarray
        Cumulative path length at each time point
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    time = trajectory[:, 2]
    
    # Calculate step displacements
    displacements = np.diff(positions, axis=0)
    
    # Calculate step lengths
    step_lengths = np.sqrt(np.sum(displacements**2, axis=1))
    
    # Calculate cumulative path length
    pl = np.zeros(len(time))
    pl[1:] = np.cumsum(step_lengths)
    
    return time, pl

In [9]:
# Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    """
    Calculate Mean Squared Displacement for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
        
    Returns:
    --------
    tau : numpy.ndarray
        Time lags
    msd : numpy.ndarray
        MSD values for each time lag
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    n_points = len(positions)
    
    # Calculate maximum tau (time lag)
    max_tau = min(n_points // 4, 1000)  # Use 1/4 of the trajectory for reliable statistics, max 1000
    
    tau = np.arange(1, max_tau + 1)
    msd = np.zeros(max_tau)
    
    # Calculate MSD for each tau
    for i, t in enumerate(tau):
        # Calculate squared displacements
        sd = np.sum((positions[t:] - positions[:-t])**2, axis=1)
        # Average to get MSD
        msd[i] = np.mean(sd)
    
    return tau, msd

In [10]:
# Turning Angle Distribution (TAD)
def calculate_tad(trajectory, bin_width=15):
    """
    Calculate Turning Angle Distribution for a trajectory
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y coordinates
    bin_width : float
        Width of the angle bins in degrees
        
    Returns:
    --------
    angle_bins : numpy.ndarray
        Center of angle bins in degrees
    tad : numpy.ndarray
        Frequency counts for each angle bin
    """
    positions = trajectory[:, :2]  # Only x, y coordinates
    
    # Calculate movement vectors between consecutive positions
    movement_vectors = np.diff(positions, axis=0)
    
    # Calculate angles of movement vectors (in radians)
    angles = np.arctan2(movement_vectors[:, 1], movement_vectors[:, 0])
    
    # Calculate turning angles (difference between consecutive angles, in radians)
    turning_angles = np.diff(angles)
    
    # Wrap angles to (-pi, pi) range
    turning_angles = np.mod(turning_angles + np.pi, 2 * np.pi) - np.pi
    
    # Convert to degrees for easier interpretation
    turning_angles_deg = np.degrees(turning_angles)
    
    # Create histogram bins
    bins = np.arange(-180, 181, bin_width)
    angle_bins = (bins[:-1] + bins[1:]) / 2  # Center of bins
    
    # Calculate histogram
    tad, _ = np.histogram(turning_angles_deg, bins=bins, density=True)
    
    return angle_bins, tad

In [11]:
class RandomWalksDashboard(param.Parameterized):
    # Parameter definitions
    rw_type = param.Selector(objects=["BM", "CRW", "LF"], default="CRW")
    num_steps = param.Integer(default=1000, bounds=(100, 5000))
    speed = param.Number(default=5.0, bounds=(1.0, 20.0))
    start_pos_x = param.Integer(default=0, bounds=(-100, 100))
    start_pos_y = param.Integer(default=0, bounds=(-100, 100))
    cauchy_coef = param.Number(default=0.7, bounds=(0.1, 0.9))
    levy_exponent = param.Number(default=1.5, bounds=(1.1, 3.0))
    metric_type = param.Selector(objects=["PL", "MSD", "TAD"], default="MSD")
    
    def __init__(self, **params):
        super(RandomWalksDashboard, self).__init__(**params)
        # Generate an initial trajectory
        self.trajectory = self.generate_trajectory()
        
        # Create initial plots
        self.trajectory_plot = pn.pane.Plotly(self.create_trajectory_plot())
        self.metric_plot = pn.pane.Plotly(self.create_metric_plot())
        
        # Create update button with callback
        self.update_button = pn.widgets.Button(name="Update", button_type="primary")
        self.update_button.on_click(self.update_plots)
    
    def generate_trajectory(self):
        """Generate trajectory based on current parameters"""
        try:
            common_params = {
                'num_steps': self.num_steps,
                'speed': self.speed,
                'start_pos_x': self.start_pos_x,
                'start_pos_y': self.start_pos_y
            }
            
            if self.rw_type == "BM":
                return generate_bm_trajectory(**common_params)
            elif self.rw_type == "CRW":
                return generate_crw_trajectory(**common_params, cauchy_coef=self.cauchy_coef)
            elif self.rw_type == "LF":
                return generate_lf_trajectory(**common_params, cauchy_coef=self.cauchy_coef, 
                                            levy_exponent=self.levy_exponent)
        except Exception as e:
            print(f"Error generating trajectory: {e}")
            # Return a simple default trajectory if there's an error
            return np.zeros((10, 3))
    
    def create_trajectory_plot(self):
        """Create 3D plot of the current trajectory"""
        try:
            fig = go.Figure()
            
            # Plot every nth point for better performance
            plot_every_n = max(1, len(self.trajectory) // 500)
            
            fig.add_trace(go.Scatter3d(
                x=self.trajectory[::plot_every_n, 0],
                y=self.trajectory[::plot_every_n, 1],
                z=self.trajectory[::plot_every_n, 2],
                mode='lines',
                name=self.rw_type,
                line=dict(color='red', width=3)
            ))
            
            fig.update_layout(
                title=f"{self.rw_type} Trajectory",
                scene=dict(
                    xaxis_title="X Position",
                    yaxis_title="Y Position",
                    zaxis_title="Time Step",
                    aspectmode='manual',
                    aspectratio=dict(x=1, y=1, z=1)
                ),
                height=600,
                width=600,
                margin=dict(l=0, r=0, b=0, t=30)
            )
            
            return fig
        except Exception as e:
            print(f"Error creating trajectory plot: {e}")
            # Return a simple empty figure
            fig = go.Figure()
            fig.update_layout(title="Error plotting trajectory")
            return fig
    
    def create_metric_plot(self):
        """Create plot for the selected metric"""
        try:
            # Calculate the selected metric
            if self.metric_type == "PL":
                time, metric_values = calculate_pl(self.trajectory)
                title = "Path Length"
                xlabel, ylabel = "Time Step", "Path Length"
                color = "green"
            elif self.metric_type == "MSD":
                time, metric_values = calculate_msd(self.trajectory)
                title = "Mean Squared Displacement"
                xlabel, ylabel = "Time Lag (τ)", "MSD"
                color = "purple"
            elif self.metric_type == "TAD":
                time, metric_values = calculate_tad(self.trajectory)
                title = "Turning Angle Distribution"
                xlabel, ylabel = "Turning Angle (degrees)", "Probability Density"
                color = "blue"
            
            # Create figure
            fig = go.Figure()
            
            fig.add_trace(go.Scatter(
                x=time,
                y=metric_values,
                mode='lines',
                name=self.metric_type,
                line=dict(color=color, width=2)
            ))
            
            fig.update_layout(
                title=title,
                xaxis_title=xlabel,
                yaxis_title=ylabel,
                height=600,
                width=600,
                margin=dict(l=0, r=0, b=0, t=30)
            )
            
            return fig
        except Exception as e:
            print(f"Error creating metric plot: {e}")
            # Return a simple empty figure
            fig = go.Figure()
            fig.update_layout(title="Error plotting metric")
            return fig
    
    def update_plots(self, event):
        """Update both plots when the button is clicked"""
        try:
            # Generate new trajectory
            self.trajectory = self.generate_trajectory()
            
            # Update plots
            self.trajectory_plot.object = self.create_trajectory_plot()
            self.metric_plot.object = self.create_metric_plot()
        except Exception as e:
            print(f"Error updating plots: {e}")
    
    @param.depends('rw_type')
    def get_parameter_panel(self):
        """Return dynamic parameter widgets based on RW type"""
        # Common parameters for all RW types
        common_params = [
            pn.Param(self.param.num_steps, widgets={'num_steps': {'type': pn.widgets.IntSlider}}),
            pn.Param(self.param.speed, widgets={'speed': {'type': pn.widgets.FloatSlider}}),
            pn.Param(self.param.start_pos_x, widgets={'start_pos_x': {'type': pn.widgets.IntSlider}}),
            pn.Param(self.param.start_pos_y, widgets={'start_pos_y': {'type': pn.widgets.IntSlider}}),
        ]
        
        # Add type-specific parameters
        if self.rw_type == "BM":
            return pn.Column(*common_params)
        elif self.rw_type == "CRW":
            return pn.Column(
                *common_params,
                pn.Param(self.param.cauchy_coef, widgets={'cauchy_coef': {'type': pn.widgets.FloatSlider}}),
            )
        elif self.rw_type == "LF":
            return pn.Column(
                *common_params,
                pn.Param(self.param.cauchy_coef, widgets={'cauchy_coef': {'type': pn.widgets.FloatSlider}}),
                pn.Param(self.param.levy_exponent, widgets={'levy_exponent': {'type': pn.widgets.FloatSlider}}),
            )
    
    def panel(self):
        """Create and return the complete dashboard layout"""
        dashboard = pn.Column(
            pn.pane.Markdown("# Random Walks Dashboard", align='center'),
            pn.Row(
                # Left column - Controls
                pn.Column(
                    pn.pane.Markdown("## RW Type", margin=(10, 5, 5, 10)),
                    pn.Param(self.param.rw_type, widgets={'rw_type': {'type': pn.widgets.Select}}),
                    pn.pane.Markdown("## Parameters", margin=(10, 5, 5, 10)),
                    self.get_parameter_panel,
                    pn.pane.Markdown("## Metric Type", margin=(10, 5, 5, 10)),
                    pn.Param(self.param.metric_type, widgets={'metric_type': {'type': pn.widgets.Select}}),
                    self.update_button,
                    width=350
                ),
                # Center column - Trajectory
                pn.Column(
                    pn.pane.Markdown("## Trajectory", margin=(10, 5, 5, 10)),
                    self.trajectory_plot,
                    width=600
                ),
                # Right column - Metric
                pn.Column(
                    pn.pane.Markdown("## Metric", margin=(10, 5, 5, 10)),
                    self.metric_plot,
                    width=600
                )
            )
        )
        return dashboard

In [12]:
# Create and display the dashboard
dashboard = RandomWalksDashboard()

In [ ]:
dashboard.panel().show()

Launching server at http://localhost:38299


gio: http://localhost:38299/: Operation not supported
